In [13]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

In [14]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""
city_code = 'BOG'
    #year-month-day
todays_date = '2022-08-20' 
#non_buyers_date = '2022-07-06' #1 months
customer_date_desde = '2022-06-20' #3 months
    #data for sku sales data
#sku_date_desde = '2022-06-06' #2 months

t_cambio = 3.88 #COL=3000 // MX=19.19 // BR=3.88

### Loading DATA_VENTAS // Removing SKUs /\ CUSTOMERS_ID

In [15]:
query = """
SELECT DISTINCT
    bo.submit_date,
    TO_CHAR(bo.submit_date, 'YYYY-mm') AS month,
    bo.order_id,
    bo.customer_id,
    boi.name,
    (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio} as gmv_usd,
    COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) as discount_applied,
    COALESCE(foi.total_tax_iva/{tipo_cambio},0) as taxes,
    boi.quantity* foi.step_unit as cant,
    UPPER(fpro.unit) as unidades,
    bs.sku_id,
    bs.addl_product_id as padre_sku_id,
    s.site_identifier_value as region_code,
    CASE
        WHEN COALESCE(bcat2.name, bcat.name) ILIKE 'Proteínas%' THEN 'Proteínas'
        ELSE COALESCE(bcat2.name, bcat.name) 
    END AS category,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    baid.adjustment_reason
FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                 fow        ON fow.owner_id = ffg.owner_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_product"               fpro       ON fpro.product_id = bp.product_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_customer"              fc         ON fc.customer_id = bo.customer_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_business_type"         bt         ON fc.business_type_id = bt.business_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat2      ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(bo.submit_date) between '{desde}' AND '{hasta}'
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value IN ('{ciudad}')
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   AND fc.business_type_id IN (1)
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(tipo_cambio=t_cambio, desde=customer_date_desde, hasta=todays_date, ciudad=city_code)

data_ventas = read_connection_data_warehouse.runQuery(query)
data_ventas[["gmv_usd", "cant"]] = data_ventas[["gmv_usd", "cant"]].astype(float)
data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [26]:
import random

a = data_ventas[data_ventas.category == 'Frutas & Verduras']['customer_id'].unique()
test = random.sample(list(a), int(len(a)*0.7))
control = [value for value in a if value not in test]

In [27]:
skus = [-300324,-300327,-300330,58813,-303252,-303249,-300531,98138,-303339,59130,-303432,59132,-303342,-303354]

In [28]:
skus_offer = "#".join([str(x) for x in skus])

In [29]:
skus_offer

'-300324#-300327#-300330#58813#-303252#-303249#-300531#98138#-303339#59130#-303432#59132#-303342#-303354'

In [30]:
global_segment_list = [{'name':'20220819_BOG_experiment_SinRestriccion', 'customersIds': list([int(x) for x in test])}]

In [31]:
csv_file = {} 
csv_file['offer_name'] = '20220819_BOG_experiment_SinRestriccion'
csv_file['offer_description'] = None
csv_file['discount'] = 11
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = '20-08-2022'
csv_file['end_date'] = '28-08-2022'
csv_file['max_uses_per_order'] = None
csv_file['max_uses_per_customer'] = None
csv_file['customer_segment_id'] = None
csv_file['sku_id'] = skus_offer
global_offer_csv = pd.DataFrame(csv_file, index=[0])


r = {}
r['CONTROL_'+'20220819_BOG_experiment_SinRestriccion'] = control
r['TEST_'+'20220819_BOG_experiment_SinRestriccion'] = test

df_test_control = pd.DataFrame.from_dict(r, orient='index').T

In [34]:
# #//--------------------------------------------
# #//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/{todays_date}_csv_{city_code}_OFFER_experiment_SinRestriccion.csv', index = False)
df_test_control.to_csv(f'SD/export_data/Below_Media/{todays_date}_csv_{city_code}_CLIENTS_experiment_SinRestriccion.csv', index = False)

with open(f'SD/export_data/Below_Media/{todays_date}_nb_{city_code}_experiment_SinRestriccion.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)    
with open(f'SD/export_data/Below_Media/{todays_date}_nb_{city_code}_experiment_SinRestriccion.json', 'w') as outfile:
        json.dump(global_segment_list, outfile)
print(f'SD/Done {city_code}')

SD/Done BOG
